In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import mysql.connector
import time
import os

try:
    #Path for chrome driver
    chromedriver_path = r'C:\Users\Priyanka\OneDrive\Desktop\DS\chromedriver\chromedriver-win64\chromedriver.exe'  

    if not os.path.exists(chromedriver_path):
        raise FileNotFoundError(f"ChromeDriver not found at {chromedriver_path}")
    service = Service(executable_path=chromedriver_path)
    
    #Accessing redbus.in website
    driver = webdriver.Chrome(service=service)
    driver.get('https://www.redbus.in/')
    time.sleep(30)
    routelist=[]
    dbroutes=[]
    
    #Loading page until the last service in the route loaded
    def load_page():
        
        prev_height = -1
        max_scrolls = 100
        scroll_count = 0
        
        
        while scroll_count < max_scrolls:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)  
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == prev_height:
                break
            prev_height = new_height
            scroll_count += 1
        return
    
    #Getting information of all the services in the particular route
    def bus_details(route):
        time.sleep(5)
        driver.get(route[0])
        time.sleep(15)
        #Scraping of government bus information
        govt_buses = driver.find_elements(By.XPATH, f"//div[contains(@class, 'group-data') and contains(@class,'clearfix')]")
        if govt_buses:
            for bus in govt_buses:
                wait = WebDriverWait(driver, 5)
                view_bus=bus.find_element(By.XPATH, f".//div[(@class='button')]")
                driver.execute_script("arguments[0].scrollIntoView(true);", view_bus)
                driver.execute_script("arguments[0].click();", view_bus)
                busname=bus.find_element(By.XPATH, f".//div[contains(@class, 'grop-name')]").text
        load_page()
        elements = driver.find_elements(By.XPATH, f"//div[contains(@class, 'clearfix') and contains(@class,'row-one')]")
        finalbuslist=[]
        for element in elements:
            row=[]
            tag_names=[]
            route_name=route[1]
            route_link=route[0]
            bus_name=element.find_element(By.XPATH, f".//div[contains(@class, 'travels')]").text
            bus_type=element.find_element(By.XPATH, f".//div[contains(@class, 'bus-type')]").text
            departing_time=element.find_element(By.XPATH, f".//div[contains(@class, 'dp-time')]").text
            duration=element.find_element(By.XPATH, f".//div[contains(@class, 'dur')]").text
            reaching_time=element.find_element(By.XPATH, f".//div[contains(@class, 'bp-time')]").text
            child_element = element.find_element(By.XPATH, f".//div[contains(@class, 'column-six')]")
            for c_element in child_element.find_elements(By.XPATH, ".//*"):
                tag_names.append(c_element.tag_name)
            if "span" in tag_names:
                starrating=element.find_element(By.XPATH, f".//span[(@class='') or contains(@class, 'icon-new-star')]").text
            else:
                starrating=""
            price=element.find_element(By.XPATH, f".//span[contains(@class, 'f-19') and contains(@class,'f-bold')]").text
            seats=element.find_element(By.XPATH, f".//div[contains(@class, 'seat-left')]").text
            row.extend([route_name,route_link,bus_name,bus_type,departing_time,duration,reaching_time,starrating,price,seats[:2]])
            finalbuslist.append(tuple(row))
        #Inserting all the scraped information of each route in the db.
        query="insert into redbusguvi values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        cursor.executemany(query,finalbuslist)
        con.commit()
        return
    
    #Starting of web scraping
    con=mysql.connector.connect(
            host="localhost",
            user="root",
            password="23051991",
            database="MDE92"
        )
    cursor=con.cursor()
    #checking whether route is in table or not
    query="select route_name from mde92.redbusguvi group by route_name"
    cursor.execute(query)
    for i in cursor:
        dbroutes.append(i)
    dbroutes = [route[0] for route in dbroutes]
    #Pagination in every state service
    pagination = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")
    for page in pagination:
        wait = WebDriverWait(driver, 5)
        driver.execute_script("arguments[0].scrollIntoView();", page)
        driver.execute_script("arguments[0].className = arguments[1];", page, 'DC_117_pageTabs DC_117_pageActive')
        driver.execute_script("arguments[0].click();", page)
        elements = driver.find_elements(By.CLASS_NAME, "route")
        for element in elements:#Collecting all the route links
            if element.text not in dbroutes:
                route_details=[]
                route_details.append(element.get_attribute('href'))
                route_details.append(element.text)
                routelist.append(route_details)
    for route in routelist: 
        bus_details(route)
    
    
    driver.quit()
except Exception as e:
    print(f"An error occurred: {e}")


An error occurred: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF7CF5FEEC2+31554]
	(No symbol) [0x00007FF7CF577EF9]
	(No symbol) [0x00007FF7CF43872A]
	(No symbol) [0x00007FF7CF40D995]
	(No symbol) [0x00007FF7CF4B44D7]
	(No symbol) [0x00007FF7CF4CC051]
	(No symbol) [0x00007FF7CF4ACDD3]
	(No symbol) [0x00007FF7CF47A33B]
	(No symbol) [0x00007FF7CF47AED1]
	GetHandleVerifier [0x00007FF7CF908B5D+3217373]
	GetHandleVerifier [0x00007FF7CF955B23+3532707]
	GetHandleVerifier [0x00007FF7CF94B120+3489184]
	GetHandleVerifier [0x00007FF7CF6AE796+750614]
	(No symbol) [0x00007FF7CF58377F]
	(No symbol) [0x00007FF7CF57EB34]
	(No symbol) [0x00007FF7CF57ECC2]
	(No symbol) [0x00007FF7CF56E18F]
	BaseThreadInitThunk [0x00007FFEE4D4257D+29]
	RtlUserThreadStart [0x00007FFEE5F4AF28+40]



In [1]:
%%writefile capstone.py
import streamlit as st
import pandas as pd
import mysql.connector

try:
    st.markdown("""
    <style>
    h1{
       text-align:center;
       color:rgb(255,75,75)
    }
    div.stButton > button{
       background-color:rgb(255,75,75)
    }
    div.stButton > button:hover {
        background-color: rgb(255,75,75);
        color: black; 
    }
    div.stButton > button:click {
        background-color: rgb(255,75,75);
        color: black; 
    }
    </style>
     """,unsafe_allow_html=True)
    st.markdown("""
    <h1>CAPSTONE</h1>
    """, unsafe_allow_html=True)
    
    #Connecting to db
    con=mysql.connector.connect(
            host="localhost",
            user="root",
            password="23051991",
            database="MDE92"
        )
    cursor=con.cursor()
    #Selecting the Route
    query="select route_name from mde92.redbusguvi group by route_name order by route_name"
    cursor.execute(query)
    results = cursor.fetchall()
    route_names=[row[0] for row in results]
    col1,col2,col3=st.columns([3,2,1])
    with col1:
        route = st.selectbox('Select the Route', route_names,key="selectbox")
    #Selecting seat type
    with col2:
        seat_type = st.selectbox('Seat type', ["Seater","Sleeper"])
    #Selecting bus type
    with col3:
        ac_type = st.selectbox('Bus type:', ["A/C","Non A/C"])
        if ac_type=="A/C":
            ac_type="A.C."
        elif ac_type=="Non A/C":
            ac_type="Non AC"
    col4,col5,col6=st.columns([5,4,6])
    #Selecting departure time
    l=[]
    for i in range(0,22):
        j=int(i)+3
        item=f"{i:02d}:00-{j:02d}:00"
        if i==21:
            l.append("21:00-23:55")
        else:
            l.append(item)
    with col4:
        dt_time = st.selectbox('Departure Time:', l)
        depart_time = dt_time.split("-")
    #Selecting rating range
    with col5:
         star_ratings = st.slider(
            'Ratings:',
            min_value=1,
            max_value=5,
            value=(1, 2),
            step=1
        )
    #Selecting price range
    with col6:
        price_range = st.slider(
            'Select a price range:',
            min_value=0,
            max_value=1000,
            value=(0, 50),
            step=10
        )

    
    #Search button
    if st.button("Search"):
        query = f"""SELECT route_name,busname,bus_type,departing_time,star_rating,price,seats_available FROM mde92.redbusguvi 
                WHERE route_name='{route}' 
                AND star_rating BETWEEN {star_ratings[0]} AND {star_ratings[1]} 
                AND price BETWEEN {price_range[0]} AND {price_range[1]}
                AND bus_type like '%{seat_type}%' 
                AND bus_type like '%{ac_type}%' 
                AND departing_time BETWEEN '{pd.to_datetime(depart_time[0]).time().strftime("%H:%M")}' AND '{pd.to_datetime(depart_time[1]).time().strftime("%H:%M")}'
                """
        cursor.execute(query)
        results=cursor.fetchall()
        if results:
            columns = [desc[0] for desc in cursor.description]
            df = pd.DataFrame(results, columns=columns)
            new_column_names = {
                "route_name": "Route",
                "busname": "Bus",
                "bus_type": "Bus Type",
                "departing_time":"Departing Time",
                "star_rating":"Rating",
                "price":"Price",
                "seats_available":"Seats available"
            }
            df.rename(columns=new_column_names, inplace=True)
            st.dataframe(df) #Creating data frame with results
        else:
            st.write("No results found")
except Exception as e:
    st.write(e)
    st.write("Some problem occured.Please try again")

Overwriting capstone.py


In [ ]:
!streamlit run capstone.py

In [1]:
l=[1,2,3,4,5]
l[-1:]

[5]